In [1]:
import sys
sys.path.append('..')

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import pandas as pd
tf.config.set_soft_device_placement(False)
tf.debugging.set_log_device_placement(True)
import numpy as np
from ampligraph.datasets import load_fb15k_237, load_yago3_10
from ampligraph.evaluation.protocol import create_mappings, to_idx

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
import time
print(tf.__version__)
assert(tf.__version__.startswith('2.5'))

from ampligraph.datasets import load_fb15k_237, load_fb13, load_fb15k, load_wn11, load_wn18, load_wn18rr, load_yago3_10
from ampligraph.latent_features import ScoringBasedEmbeddingModel

2.5.0-dev20201208


In [2]:
dataset = load_fb15k_237()

Jump to 
- [Partitioned](#Training/eval-with-partition)
- [Discovery](#Discovery)

# Train/eval without partition

In [3]:
optim = tf.optimizers.Adam(learning_rate=0.01)
# optim = 'adam'

# loss = nll
# loss = 'self_adversarial'
from ampligraph.latent_features.loss_functions import SelfAdversarialLoss, NLLMulticlass
loss = SelfAdversarialLoss({'margin': 0.1, 'alpha': 5, 'reduction': 'sum'})
loss = NLLMulticlass({'reduction': 'sum'})
model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300,
                                     scoring_type='ComplEx')



model.compile(optimizer=optim, loss=loss)

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

dataset = load_fb15k_237()

start = time.time()
model.fit(dataset['train'],
             batch_size=10000,
             epochs=10,
             validation_freq=50,
             validation_batch_size=100,
             validation_data = dataset['valid'],
         callbacks=[checkpoint])

end = time.time()
print('Time taken:', end-start)

Epoch 1/10
29/29 [==============================] - 2s 65ms/step - loss: 16389.1543

Epoch 00001: loss improved from inf to 16389.15430, saving model to ./chkpt1
Epoch 2/10
29/29 [==============================] - 1s 21ms/step - loss: 10464.8750

Epoch 00002: loss improved from 16389.15430 to 10464.87500, saving model to ./chkpt1
Epoch 3/10
29/29 [==============================] - 1s 21ms/step - loss: 7459.9829

Epoch 00003: loss improved from 10464.87500 to 7459.98291, saving model to ./chkpt1
Epoch 4/10
29/29 [==============================] - 1s 20ms/step - loss: 5856.6294

Epoch 00004: loss improved from 7459.98291 to 5856.62939, saving model to ./chkpt1
Epoch 5/10
29/29 [==============================] - 1s 21ms/step - loss: 4870.1465

Epoch 00005: loss improved from 5856.62939 to 4870.14648, saving model to ./chkpt1
Epoch 6/10
29/29 [==============================] - 1s 20ms/step - loss: 4196.1782

Epoch 00006: loss improved from 4870.14648 to 4196.17822, saving model to ./chkpt1

In [4]:
ranks = model.evaluate(np.array([['/m/01cr28', '/location/country/form_of_government', '/m/02lkcc'],
                     ['/m/07tw_b', '/location/country/form_of_government', '/m/02lkcc'],
                     ['/m/073tm9', '/location/country/form_of_government', '/m/02lkcc']]), 
                       use_filter={'train': dataset['train']}, 
                       corrupt_side='s,o', 
                       verbose=True)

2/2 [==============================] - 2s 888ms/step


In [5]:
ranks

array([[11594, 13650],
       [ 5554,  6117],
       [11965, 12250]], dtype=int32)

In [6]:
start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
                       entities_subset=['/m/08966', '/m/05lf_', '/m/0f8l9c', '/m/04ghz4m'],
                      
                      use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
mrr_score(ranks) # will give very high mrr


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 54s 260ms/step


0.964608816909678

In [8]:
start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
                      use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
mrr_score(ranks)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 67s 327ms/step


0.22008795412679102

In [9]:
start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
                       entities_subset=list(model.data_indexer.backend.entities_dict.values()),
                      
                      use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mrr_score(ranks)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 70s 337ms/step
Time taken: 69.58674621582031


0.22008795412679102

In [10]:
np.random.seed(0)
pred = model.predict('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100)
print(pred.shape)
print(np.sort(pred))


28 triples containing invalid keys skipped!
(20438,)
[-2.6117408 -2.5582652 -2.4347572 ... 30.218775  31.553251  41.351562 ]


In [11]:
model.calibrate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                batch_size=10000, positive_base_rate=0.9, epochs=100)


28 triples containing invalid keys skipped!


In [12]:

out = model.predict_proba('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', batch_size=10000)
print(np.sort(out))
print(np.argsort(out))
pred_out = model.predict('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt')
print(np.sort(pred_out))
print(np.argsort(pred_out))



28 triples containing invalid keys skipped!
[0.41542763 0.41815612 0.42447674 ... 0.9985715  0.9989201  0.9998617 ]
[ 4066   611 18634 ...   990 10437 14612]

28 triples containing invalid keys skipped!
[-2.6117408 -2.5582652 -2.4347572 ... 30.218775  31.553251  41.351562 ]
[ 4066   611 18634 ...   990 10437 14612]


In [13]:

start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 68s 328ms/step
Time taken: 67.6325569152832


(298.09800371856346,
 0.22008795412679102,
 0.13516488893238085,
 0.3928711224190234,
 20438)

In [14]:
model.save_weights('./calibrated_model')

## Load Checkpoint and evaluate

In [15]:
start = time.time()
loaded_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='ComplEx')
loaded_model.load_weights('./calibrated_model')
ranks = loaded_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 67s 324ms/step
Time taken: 66.87737703323364


(298.09800371856346,
 0.22008795412679102,
 0.13516488893238085,
 0.3928711224190234,
 20438)

In [16]:
pred = loaded_model.predict('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100)
print(pred.shape)
print(pred)


28 triples containing invalid keys skipped!
(20438,)
[14.611031    9.369465    7.550661   ...  5.5516806   0.93437165
  5.4705043 ]


In [17]:
out = loaded_model.predict_proba('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', batch_size=10000)
np.argsort(out), np.argsort(loaded_model.predict('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt')), np.sort(out)


28 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!


(array([ 4066,   611, 18634, ...,   990, 10437, 14612]),
 array([ 4066,   611, 18634, ...,   990, 10437, 14612]),
 array([0.41542763, 0.41815612, 0.42447674, ..., 0.9985715 , 0.9989201 ,
        0.9998617 ], dtype=float32))

# Training/eval with partition 

## Training with RandomEdges partitioner

In [18]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY
dataset_loader = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt', 
                                 backend=SQLiteAdapter,
                                    batch_size=1000, 
                                    dataset_type='train', 
                                     use_filter=False,
                                    use_indexer=True)

In [19]:
# Choose the partitioner 
partitioner = PARTITION_ALGO_REGISTRY.get('RandomEdges')(dataset_loader, k=3)


_split: memory before: 896.0Bytes, after: 12.748MB, consumed: 12.747MB; exec time: 31.495s


In [20]:

optim = tf.optimizers.Adam(learning_rate=0.001, amsgrad=True)

partitioned_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')
partitioned_model.compile(optimizer=optim, loss='multiclass_nll')

In [21]:

start = time.time()
partitioned_model.fit(partitioner,
                     batch_size=1000, use_partitioning=True,             
                     epochs=10)
print((time.time()-start))

Epoch 1/10
274/274 [==============================] - 19s 68ms/step - loss: 1587.2365
Epoch 2/10
274/274 [==============================] - 19s 69ms/step - loss: 1413.6970
Epoch 3/10
274/274 [==============================] - 20s 71ms/step - loss: 1251.7213
Epoch 4/10
274/274 [==============================] - 19s 68ms/step - loss: 1116.2098
Epoch 5/10
274/274 [==============================] - 19s 71ms/step - loss: 1007.5591
Epoch 6/10
274/274 [==============================] - 19s 69ms/step - loss: 920.2757
Epoch 7/10
274/274 [==============================] - 19s 70ms/step - loss: 849.2607
Epoch 8/10
274/274 [==============================] - 20s 71ms/step - loss: 790.6967
Epoch 9/10
274/274 [==============================] - 19s 70ms/step - loss: 741.4282
Epoch 10/10
274/274 [==============================] - 20s 71ms/step - loss: 699.5930
204.38951015472412


In [22]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=SQLiteAdapter,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        use_indexer=partitioned_model.data_handler.get_mapper())


28 triples containing invalid keys skipped!


In [23]:

start = time.time()
ranks = partitioned_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)

53/53 [==============================] - 130s 2s/step
Time taken: 130.39389443397522


(464.99716214893823, 0.09894986543567783, 0.0, 0.2728251296604364, 20438)

## Save and Load

In [24]:
partitioned_model.save_weights('./best_model')

In [25]:
loaded_part_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')

loaded_part_model.load_weights('./best_model')

In [26]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=DummyBackend,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        use_indexer=loaded_part_model.data_indexer)


28 triples containing invalid keys skipped!


In [27]:


start = time.time()
ranks = loaded_part_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)

53/53 [==============================] - 131s 2s/step
Time taken: 130.80136060714722


(464.99716214893823, 0.09894986543567783, 0.0, 0.2728251296604364, 20438)

# Training/eval with partition (default Partitioning Approach)

In [28]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY


In [29]:
optim = tf.optimizers.Adam(learning_rate=0.0001, amsgrad=True)

partitioned_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')
partitioned_model.compile(optimizer=optim, loss='multiclass_nll')


start = time.time()
partitioned_model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                     batch_size=1000, use_partitioning=True,
                     epochs=10)
print((time.time()-start))

_split: memory before: 102.46MB, after: 115.18MB, consumed: 12.722MB; exec time: 96.584s
Epoch 1/10
277/277 [==============================] - 12s 44ms/step - loss: 1765.2789
Epoch 2/10
277/277 [==============================] - 12s 43ms/step - loss: 1750.7335
Epoch 3/10
277/277 [==============================] - 12s 45ms/step - loss: 1739.2776
Epoch 4/10
277/277 [==============================] - 12s 44ms/step - loss: 1728.5615
Epoch 5/10
277/277 [==============================] - 12s 44ms/step - loss: 1718.0000
Epoch 6/10
277/277 [==============================] - 12s 44ms/step - loss: 1707.3138
Epoch 7/10
277/277 [==============================] - 12s 43ms/step - loss: 1696.3335
Epoch 8/10
277/277 [==============================] - 12s 44ms/step - loss: 1685.0465
Epoch 9/10
277/277 [==============================] - 12s 44ms/step - loss: 1673.4987
Epoch 10/10
277/277 [==============================] - 12s 44ms/step - loss: 1661.8308
227.05004739761353


In [30]:
len(set(dataset['train'][:, 0]).union(set(dataset['train'][:, 2])))

14505

In [31]:


start = time.time()
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)


28 triples containing invalid keys skipped!
53/53 [==============================] - 133s 3s/step
Time taken: 133.19726157188416


(1852.678613367257, 0.08854410426360261, 0.0, 0.2376944906546629, 20438)

## Filtered evaluation

In [32]:
start = time.time()
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=400,
                       corrupt_side='s,o',
                        use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                              'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                              'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
53/53 [==============================] - 333s 6s/step
Time taken: 333.0659215450287


(1724.8188912809474,
 0.1961136583183794,
 0.1436539778843331,
 0.3027448869752422,
 20438)

# Discovery

### Discover Facts

In [3]:
from ampligraph.discovery import discover_facts

model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300,
                                     scoring_type='ComplEx')



model.compile(optimizer='adam', loss='multiclass_nll')

start = time.time()
model.fit(dataset['train'],
             batch_size=10000,
             epochs=10,
             validation_freq=50,
             validation_batch_size=100,
             validation_data = dataset['valid'])

end = time.time()
print('Time taken:', end-start)

discover_facts(dataset['train'][:100], 
               model, 
               top_n=100, 
               strategy='random_uniform', 
               max_candidates=100, 
               target_rel='/location/country/form_of_government', 
               seed=0)

Epoch 1/10
29/29 [==============================] - 2s 64ms/step - loss: 17412.5391
Epoch 2/10
29/29 [==============================] - 1s 21ms/step - loss: 17398.1309
Epoch 3/10
29/29 [==============================] - 1s 21ms/step - loss: 17296.1465
Epoch 4/10
29/29 [==============================] - 1s 20ms/step - loss: 16878.4785
Epoch 5/10
29/29 [==============================] - 1s 21ms/step - loss: 15909.9961
Epoch 6/10
29/29 [==============================] - 1s 20ms/step - loss: 14546.6768
Epoch 7/10
29/29 [==============================] - 1s 21ms/step - loss: 13176.4453
Epoch 8/10
29/29 [==============================] - 1s 20ms/step - loss: 11976.7051
Epoch 9/10
29/29 [==============================] - 1s 21ms/step - loss: 10962.2529
Epoch 10/10
29/29 [==============================] - 1s 21ms/step - loss: 10104.8604
Time taken: 8.017615556716919


(array([['/m/0fvf9q', '/location/country/form_of_government', '/m/0m313']],
       dtype=object),
 array([36.]))

### Find Clusters

In [4]:
import requests
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns


from ampligraph.datasets import load_from_csv
from ampligraph.discovery import find_clusters

# International football matches triples
# See tutorial here to understand how the triples are created from a tabular dataset:
url = 'https://ampligraph.s3-eu-west-1.amazonaws.com/datasets/football.csv'
open('football.csv', 'wb').write(requests.get(url).content)
X = load_from_csv('.', 'football.csv', sep=',')[:, 1:]

model = ScoringBasedEmbeddingModel(eta=5, 
                                 k=300,
                                 scoring_type='ComplEx')



model.compile(optimizer='adam', loss='multiclass_nll')

model.fit(X,
          batch_size=10000,
          epochs=10)

df = pd.DataFrame(X, columns=["s", "p", "o"])
teams = np.unique(np.concatenate((df.s[df.s.str.startswith("Team")],
                               df.o[df.o.str.startswith("Team")])))
team_embeddings = model.get_embeddings(teams, embedding_type='e')

embeddings_2d = PCA(n_components=2).fit_transform(np.array([i for i in team_embeddings]))

# Find clusters of embeddings using KMeans

kmeans = KMeans(n_clusters=6, n_init=100, max_iter=500)
clusters = find_clusters(teams, model, kmeans, mode='e')


Epoch 1/10
46/46 [==============================] - 2s 41ms/step - loss: 17856.8047
Epoch 2/10
46/46 [==============================] - 1s 24ms/step - loss: 17844.0449
Epoch 3/10
46/46 [==============================] - 1s 24ms/step - loss: 17730.1875: 0s - loss: 17800.8 - ETA: 0s - loss: 17774.0
Epoch 4/10
46/46 [==============================] - 1s 25ms/step - loss: 17330.3867
Epoch 5/10
46/46 [==============================] - 1s 24ms/step - loss: 16672.2637
Epoch 6/10
46/46 [==============================] - 1s 24ms/step - loss: 15780.3242
Epoch 7/10
46/46 [==============================] - 1s 25ms/step - loss: 14677.3369
Epoch 8/10
46/46 [==============================] - 1s 25ms/step - loss: 13518.6426
Epoch 9/10
46/46 [==============================] - 1s 25ms/step - loss: 12447.1543
Epoch 10/10
46/46 [==============================] - 1s 25ms/step - loss: 11510.4658


### Find Duplicates

In [5]:
import pandas as pd
import numpy as np
import re

# The IMDB dataset used here is part of the Movies5 dataset found on:
# The Magellan Data Repository (https://sites.google.com/site/anhaidgroup/projects/data)
import requests
url = 'http://pages.cs.wisc.edu/~anhai/data/784_data/movies5.tar.gz'
open('movies5.tar.gz', 'wb').write(requests.get(url).content)
import tarfile
tar = tarfile.open('movies5.tar.gz', "r:gz")
tar.extractall()
tar.close()

# Reading tabular dataset of IMDB movies and filling the missing values
imdb = pd.read_csv("movies5/csv_files/imdb.csv")
imdb["directors"] = imdb["directors"].fillna("UnknownDirector")
imdb["actors"] = imdb["actors"].fillna("UnknownActor")
imdb["genre"] = imdb["genre"].fillna("UnknownGenre")
imdb["duration"] = imdb["duration"].fillna("0")

# Creating knowledge graph triples from tabular dataset
imdb_triples = []

for _, row in imdb.iterrows():
    movie_id = "ID" + str(row["id"])
    directors = row["directors"].split(",")
    actors = row["actors"].split(",")
    genres = row["genre"].split(",")
    duration = "Duration" + str(int(re.sub("\D", "", row["duration"])) // 30)

    directors_triples = [(movie_id, "hasDirector", d) for d in directors]
    actors_triples = [(movie_id, "hasActor", a) for a in actors]
    genres_triples = [(movie_id, "hasGenre", g) for g in genres]
    duration_triple = (movie_id, "hasDuration", duration)

    imdb_triples.extend(directors_triples)
    imdb_triples.extend(actors_triples)
    imdb_triples.extend(genres_triples)
    imdb_triples.append(duration_triple)

# Training knowledge graph embedding with ComplEx model
model = ScoringBasedEmbeddingModel(eta=5, 
                             k=300,
                             scoring_type='ComplEx')



model.compile(optimizer='adam', loss='multiclass_nll')
imdb_triples = np.array(imdb_triples)
model.fit(imdb_triples,
      batch_size=10000,
      epochs=20)

# Finding duplicates movies (entities)
from ampligraph.discovery import find_duplicates

entities = np.unique(imdb_triples[:, 0])
dups, _ = find_duplicates(entities, model, mode='e', tolerance=0.45)
id_list = []
for data in dups:
    for i in data:
        id_list.append(int(i[2:]))
print(imdb.iloc[id_list[:6]][['movie_name', 'year']])

Epoch 1/20
7/7 [==============================] - 1s 134ms/step - loss: 15612.8799
Epoch 2/20
7/7 [==============================] - 0s 22ms/step - loss: 15610.5010
Epoch 3/20
7/7 [==============================] - 0s 21ms/step - loss: 15607.7412
Epoch 4/20
7/7 [==============================] - 0s 20ms/step - loss: 15604.0674
Epoch 5/20
7/7 [==============================] - 0s 22ms/step - loss: 15598.9365
Epoch 6/20
7/7 [==============================] - 0s 22ms/step - loss: 15591.7188
Epoch 7/20
7/7 [==============================] - 0s 20ms/step - loss: 15581.6055
Epoch 8/20
7/7 [==============================] - 0s 21ms/step - loss: 15567.6807
Epoch 9/20
7/7 [==============================] - 0s 20ms/step - loss: 15548.8184
Epoch 10/20
7/7 [==============================] - 0s 19ms/step - loss: 15523.8721
Epoch 11/20
7/7 [==============================] - 0s 21ms/step - loss: 15491.4668
Epoch 12/20
7/7 [==============================] - 0s 19ms/step - loss: 15450.2480
Epoch 13/20


### Query TopN

In [3]:
import requests
from ampligraph.datasets import load_from_csv
from ampligraph.discovery import discover_facts
from ampligraph.discovery import query_topn

# Game of Thrones relations dataset
url = 'https://ampligraph.s3-eu-west-1.amazonaws.com/datasets/GoT.csv'
open('GoT.csv', 'wb').write(requests.get(url).content)
X = load_from_csv('.', 'GoT.csv', sep=',')

model = ScoringBasedEmbeddingModel(eta=5, 
                             k=150,
                             scoring_type='TransE')



model.compile(optimizer='adagrad', loss='pairwise')
model.fit(X,
      batch_size=100,
      epochs=20)
query_topn(model, top_n=5,
        head='Eddard Stark', relation='ALLIED_WITH', tail=None,
        ents_to_consider=None, rels_to_consider=None)


Epoch 1/20
33/33 [==============================] - 1s 34ms/step - loss: 491.7411
Epoch 2/20
33/33 [==============================] - 0s 4ms/step - loss: 486.8790
Epoch 3/20
33/33 [==============================] - 0s 4ms/step - loss: 483.2306
Epoch 4/20
33/33 [==============================] - 0s 4ms/step - loss: 480.0301
Epoch 5/20
33/33 [==============================] - 0s 4ms/step - loss: 477.2192
Epoch 6/20
33/33 [==============================] - 0s 5ms/step - loss: 474.6082
Epoch 7/20
33/33 [==============================] - 0s 4ms/step - loss: 472.1904
Epoch 8/20
33/33 [==============================] - 0s 5ms/step - loss: 469.9152
Epoch 9/20
33/33 [==============================] - 0s 4ms/step - loss: 467.8103
Epoch 10/20
33/33 [==============================] - 0s 4ms/step - loss: 465.7492
Epoch 11/20
33/33 [==============================] - 0s 4ms/step - loss: 463.8291
Epoch 12/20
33/33 [==============================] - 0s 4ms/step - loss: 461.9291
Epoch 13/20
33/33 [=====

(array([['Eddard Stark', 'ALLIED_WITH', 'House Frey of the Crossing'],
        ['Eddard Stark', 'ALLIED_WITH', 'House Stark of Winterfell'],
        ['Eddard Stark', 'ALLIED_WITH',
         "House Targaryen of King's Landing"],
        ['Eddard Stark', 'ALLIED_WITH',
         'House Lannister of Casterly Rock'],
        ['Eddard Stark', 'ALLIED_WITH', 'House Greyjoy of Pyke']],
       dtype='<U44'),
 array([-1.0268682, -1.0519509, -1.0546845, -1.1503534, -1.1699506],
       dtype=float32))